In [1]:
import os
from urllib.request import urlretrieve
from multiprocessing import Pool
from collections import defaultdict, namedtuple
from pprint import pprint
import tqdm
import vk_api
from vk_api.audio import VkAudio
import json
import logging
import sys
from urllib.request import urlretrieve
from DTask import DTask
from DTasksProcessor import DTaskProcessor


In [2]:

API_VERSION = '5.87'

flags = {'creds_path':'creds.json', 
        'save_path':'dumps/audio'}


logger =logging.getLogger(__name__)
logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

def get_password_and_id(cp):
    creds = json.load(open(cp))
    return creds['pass'], creds['id']

password, id = get_password_and_id(flags['creds_path'])

session = vk_api.VkApi(app_id='6787646', login=id,
                         scope='audio', password=password, api_version=API_VERSION)
session.auth()
vkapi = session.get_api()
audioapi = vk_api.audio.VkAudio(session)

2019-03-05 12:39:38,755 | INFO : Auth with login: 89036004477
2019-03-05 12:39:38,759 | INFO : Checking remixsid...
2019-03-05 12:39:39,707 | INFO : remixsid is valid
2019-03-05 12:39:39,708 | INFO : Checking security check request
2019-03-05 12:39:40,360 | INFO : Security check is not required
2019-03-05 12:39:41,200 | INFO : access_token from config is valid


In [12]:
def dtask_from_audio(obj):
    url = obj['url']
    name = '{} {}.mp3'.format(obj.get('artist', ''), obj.get('title',''))
    name = name.replace('/','')
    size = obj.get('duration',100)
    
    
    task = DTask(url, flags['save_path'],size = size, filename = name)
    return task

In [13]:
tasks = []
for a in audioapi.get_iter(owner_id=-43695482):
    tasks.append(dtask_from_audio(a))

In [15]:
tp = DTaskProcessor(tasks)
tp.process_tasks()

 50%|█████     | 8/16 [00:12<00:09,  1.15s/it]


[]

In [14]:
for t in tasks:
    print(t.should_be_executed, os.path.exists(t.full_name), t.full_name, t.error)

True False dumps/audio/Шкловский Жрать культуру (Это не вам2017).mp3 None
True False dumps/audio/Шкловский Бассейн (Это не вам2017).mp3 None
True False dumps/audio/Шкловский Метрополитен (Это не вам2017).mp3 None
True False dumps/audio/Шкловский Принцесса (Это не вам2017).mp3 None
True False dumps/audio/Шкловский Брат (Это не вам2017).mp3 None
True False dumps/audio/Шкловский Еду по Москве (Это не вам2017).mp3 None
True False dumps/audio/Шкловский Местечкис (Это не вам2017).mp3 None
True False dumps/audio/Шкловский Листовки (Это не вам2017).mp3 None
False True dumps/audio/Шкловский Девяностые.mp3 None
False True dumps/audio/Шкловский Могуч.mp3 None
False True dumps/audio/Шкловский Племянница.mp3 None
False True dumps/audio/Шкловский Люся.mp3 None
False True dumps/audio/Шкловский 13 лет.mp3 None
False True dumps/audio/Шкловский Великан.mp3 None
False True dumps/audio/Шкловский ОГПУ.mp3 None
False True dumps/audio/Шкловский Кот.mp3 None


In [8]:
urlretrieve(tasks[0].url, tasks[0].full_name)



FileNotFoundError: [Errno 2] No such file or directory: 'dumps/audio/Шкловский Жрать культуру (Это не вам/2017).mp3'

In [11]:
tasks[0].full_name.replace('/','')

'dumpsaudioШкловский Жрать культуру (Это не вам2017).mp3'

In [20]:
q = 'hoizer'
for i in audioapi.search(q, count=50):
    print(i)

{'id': 71242763, 'owner_id': 11194224, 'url': 'https://psv4.vkuseraudio.net/c1623/u13040156/audios/7f3919018cbd.mp3?extra=280P_EtRZ3C0B9UaJ4ETa4fV4lkH3QNFbeZylFRYnujBHnprd9fVkmZg4xxLgRuSVoGRiBNUAj4Kg15fZuo8cVx_9gMI7c8iuMw_4GxbsDQp9IkNIx34P3zcjNX1CW-P2f9DxJrQdRTy04POiTJ6Ug', 'artist': 'Бард "Сигурд" ', 'title': 'Завесельная', 'duration': 375}
{'id': 456239030, 'owner_id': 50415897, 'url': 'https://cs9-22v4.vkuseraudio.net/p11/4428c144e4f93a.mp3?extra=1-ml3dwodz9u8CE3-N0voUIePieJYsmccSAqqGqJIXbBw56jkOMGuJlYs0i3-PKBdsCJ29ZB7IymwxZnFX_yR0BwDQoODiCfIkrfdO9jU5UNOKViFx3_P17LbLyaJDkJJD0Fqh91LocnXx60w_HgHmQ', 'artist': 'Бард Сигурд', 'title': 'О! Всё сгорело нахуй!', 'duration': 135}
{'id': 456239910, 'owner_id': 347669103, 'url': 'https://psv4.vkuseraudio.net/c422423/u155061237/audios/d6912a2e9a4e.mp3?extra=2EDviyESFogtEtrHXfO5hK7EhQbQ3pictsOvK99Y0RM4b7QofcBrANKps0YbRGnHYT84X_qHJ_iR-I73NCjNesHlfFET0dUTFBmHHlFuYhx6409tfsMZ413j1GFOz-Z_cRKUmIq7_KIPUVMKHN8I1A61', 'artist': 'Константин День Сверлов